In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install idx2numpy

import tensorflow.keras as keras
import idx2numpy
import cv2 as cv
import numpy as np
import gzip
import pickle
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from   tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


train_data_path = '/content/gdrive/My Drive/data1/train-images-idx3-ubyte.gz'
test_data_path = '/content/gdrive/My Drive/data1/t10k-images-idx3-ubyte.gz'
train_label_path = '/content/gdrive/My Drive/data1/train-labels-idx1-ubyte.gz'
test_label_path = '/content/gdrive/My Drive/data1/t10k-labels-idx1-ubyte.gz'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
class MiniBatch_GD():

	def __init__(self):
		 self.network = keras.Sequential()
		 self.network.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(28,28,1)))
		 self.network.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
		 self.network.add(keras.layers.Conv2D(64, kernel_size=3, activation='relu'))
		 self.network.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None))
		 self.network.add(keras.layers.Flatten())
		 self.network.add(keras.layers.Dense(10, activation='softmax'))

	def extract_data(self, filepath, num_img):
		with gzip.open(filepath) as f:
			f.read(16)
			buf = f.read(28 * 28 * num_img)
			train_data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
			train_data = train_data.reshape(num_img, 28, 28, 1)
			return train_data

	def extract_labels(self, filepath, num_img):
		with gzip.open(filepath) as f:
			f.read(8)
			buf = f.read(num_img)
			labels= np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
			# labels = labels.reshape(num_img, 28, 28, 1)
			return labels


In [0]:
mbgd = MiniBatch_GD()

train_data = mbgd.extract_data(train_data_path, 60000)
train_labels = mbgd.extract_labels(train_label_path, 60000)
test_data = mbgd.extract_data(test_data_path, 10000)
test_labels = mbgd.extract_labels(test_label_path, 10000)

Fd =open('/content/gdrive/My Drive/data1/Network_Best_one.h5', 'w+') 
Fd.close()

In [51]:
train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)
opt = keras.optimizers.SGD(lr = 0.001 , momentum = 0.9)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=10 )
cs= ModelCheckpoint(filepath='/content/gdrive/My Drive/data1/Network_Best_one.h5', monitor='val_loss', save_best_only=True)
mbgd.network.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
hist = mbgd.network.fit(train_data, train_labels, batch_size = 170, validation_data=(test_data, test_labels), epochs=300,callbacks=[es,cs])
mbgd.network.save('/content/gdrive/My Drive/data1/Network_config_one.h5')
with open('/content/gdrive/My Drive/data1/History_config_one', 'wb') as file:
  pickle.dump(hist.history, file)



print("OUTPUT FROM BEST MODEL")
#Bestmodel = MiniBatch_GD()
mbgd.network.load_weights('/content/gdrive/My Drive/data1/Network_Best_one.h5')
oss, acc = mbgd.network.evaluate(test_data,  test_labels, verbose=2)



Train on 60000 samples, validate on 10000 samples
Epoch 1/300
60000/60000 [==============================] - 3s 50us/sample - loss: 1.3775 - acc: 0.6034 - val_loss: 0.6990 - val_acc: 0.7557
Epoch 2/300
60000/60000 [==============================] - 2s 34us/sample - loss: 0.6345 - acc: 0.7754 - val_loss: 0.6167 - val_acc: 0.7814
Epoch 3/300
60000/60000 [==============================] - 2s 34us/sample - loss: 0.5610 - acc: 0.7976 - val_loss: 0.5541 - val_acc: 0.7970
Epoch 4/300
60000/60000 [==============================] - 2s 35us/sample - loss: 0.5067 - acc: 0.8180 - val_loss: 0.5156 - val_acc: 0.8100
Epoch 5/300
60000/60000 [==============================] - 2s 37us/sample - loss: 0.4807 - acc: 0.8282 - val_loss: 0.4993 - val_acc: 0.8227
Epoch 6/300
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4612 - acc: 0.8339 - val_loss: 0.4928 - val_acc: 0.8180
Epoch 7/300
60000/60000 [==============================] - 2s 34us/sample - loss: 0.4487 - acc: 0.8385 - val_l

In [2]:




import matplotlib.pyplot as plt

plt.plot(hist.history['acc'])
plt.title('Model accuracy (higher is better)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

NameError: ignored